## Objective

Since what I'm most interested in is which factors are most associated with whether an issue is unresolved, random forests seems like the most appropriate model choice, since I can look at the variable importance plot.

Logistic regression is a close second; random forests usually have better performance.

Random forests can also do a better job at getting signal out of raw lat and long. For logistic regression, there would need to be a linear relationship between lat and long and the log-odds of the issue being unresolved, which is a taller order.

Thinking about my engineered feature `days_from_feb_2016` for example, Logistic Regression wouldn't do a good job with it, but a tree-based model would.

In [1]:
from __future__ import division
import pandas as pd
from datetime import timedelta, datetime

In [2]:
import warnings
import seaborn as sns

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")

from pylab import rcParams
rcParams['figure.figsize'] = 20, 5

%matplotlib inline

In [3]:
df = pd.read_pickle('../data/data_w_transformed_census_and_removed_invalid_rows_and_cols.pkl')
df.shape

(716310, 36)

In [ ]:
df.head(1).T

## Preprocessing

In [ ]:
df.TARGET_DT.head(100).isnull().sum()

In [ ]:
df.LOCATION_ZIPCODE.head(100).isnull().sum()

In [4]:
df['days_from_feb_2016'] = (datetime(year=2016, month=2, day=1) - df.OPEN_DT).map(lambda x: x.days)
df['LOCATION_ZIPCODE'] = df['LOCATION_ZIPCODE'].astype('object').fillna('other')
df['neighborhood'] = df['neighborhood'].fillna('other')
df['is_snow_in_type'] = df.TYPE.map(lambda txt: 'snow' in txt.lower())
# df['race_majority'] = df[[i for i in df.columns if 'race' in i]].idxmax(axis=1) # not useful bc will be dummified

In [5]:
df = df.drop(
    ['OPEN_DT',
     'TARGET_DT',
     'CLOSED_DT',
     'COMPLETION_TIME',
     'Property_ID',
     'LOCATION_STREET_NAME',
     'CASE_TITLE',
     'CASE_ENQUIRY_ID',
     'SUBJECT',
     'Department',
     'tract_and_block_group'
    ], 
    axis=1
)

In [6]:
df.shape

(716310, 27)

In [7]:
df.head(1).T

,0
REASON,Signs & Signals
TYPE,Sign Repair
SubmittedPhoto,False
neighborhood,Downtown / Financial District
LOCATION_ZIPCODE,other
Property_Type,Intersection
LATITUDE,42.3537
LONGITUDE,-71.058
Source,Constituent Call
race_white,0.77619


### Dummifying bc sklearn's random forest implementation doesn't accept string values

In [8]:
def dummify(df, column):
    # from Darren's linear regression slides
    print '{} is your baseline'.format(sorted(df[column].unique())[-1])
    dummy = pd.get_dummies(df[column]).rename(columns=lambda x: column+'_'+str(x)).iloc[:,0:len(df[column].unique())-1]
    df = df.drop(column,axis=1) #Why not inplace? because if we do inplace, it will affect the df directly
    return pd.concat([df,dummy],axis=1)

In [9]:
df1 = dummify(df, 'TYPE')
df2 = dummify(df1, 'neighborhood')
df3 = dummify(df2, 'LOCATION_ZIPCODE')
df4 = dummify(df3, 'Property_Type')
df5 = dummify(df4, 'Source')
df6 = dummify(df5, 'school')
df7 = dummify(df6, 'housing')

Zoning is your baseline
other is your baseline
other is your baseline
Intersection is your baseline
Twitter is your baseline
8_6th_grade is your baseline
rent is your baseline


In [10]:
df6.shape

(716310, 302)

## OK, let's put it into the model!

In [11]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Let's split the data first between train and test, 80/20.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('is_issue_unresolved', axis=1), 
    df.is_issue_unresolved, 
    test_size=0.2, 
    random_state=300
)

In [13]:
pipe = make_pipeline(
    RandomForestClassifier(
        bootstrap=True, 
        class_weight='balanced_subsample', 
        n_estimators=10
    )
)

In [ ]:
params = {'randomforestclassifier__max_depth': [10, 20],
      'randomforestclassifier__min_samples_leaf': [101, 1001], # odd for bin class, to avoid ties, 1% of num_rows for max bound
}

model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=5, verbose=True)
model.fit(X_train, y_train);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/pool.py", line 326, in _handle_workers
    pool._maintain_pool()
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/pool.py", line 230, in _maintain_pool
    self._repopulate_pool()
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/pool.py", line 223, in _repopulate_pool
    w.start()
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 130, in start
    self._popen = Popen(self)
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/forking.py", line 121, in __init__
    self.pid = os.fork()
OSError: [Errno 12] Cannot allocate memory



In [ ]:
pd.DataFrame(model.cv_results)

If my best model included 40 trees, was that enough?

In [ ]:
model.cv_results_['params']

Let's make variable importance graph.

A downside of this graph is that highly correlated features will split importance.

## Using Logistic Regression with L2 Lasso Regularization

which is my favorite way to do feature subset selection at the moment :)

There are probably issues when features are co-linear. I'll need to read up on this.

In [ ]:
# http://nbviewer.jupyter.org/github/JWarmenhoven/ISLR-python/blob/master/Notebooks/Chapter%206.ipynb#6.6.2-The-Lasso
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso(max_iter=10000)
coefs = []

for a in alphas*2:
    lasso.set_params(alpha=a)
    lasso.fit(scale(X_train), y_train)
    coefs.append(lasso.coef_)

ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Lasso coefficients as a function of the regularization');